# Assignment 5

Build CNN model for sentiment analysis (binary classification) of IMDB Reviews (https://www.kaggle.com/utathya/imdb-review-dataset).
You can use data with label="unsup" for pretraining of embeddings. Here you are forbidden to use test dataset for pretraining of embeddings.  
Your quality metric is accuracy score on test dataset. Look at "type" column for  train/test split.  
You can use pretrained embeddings from external sources.  
You have to provide data for trials with different hyperparameter values.  

You have to beat following baselines:  
[3 points] acc = 0.75  
[5 points] acc = 0.8  
[8 points] acc = 0.9  

[2 points] for using unsupervised data  

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/ml_4year/hse_seminars/2019/seminar_8

/content/drive/My Drive/ml_4year/hse_seminars/2019/seminar_8


## Pre-training Word2Vec Embeddings

In [0]:
import pandas as pd

In [0]:
with open('imdb_master.csv', 'r', encoding='utf-8', errors="ignore") as inp:
  csv_string = inp.read()

with open('imdb_master_clean.csv', 'w', encoding='utf-8') as outp:
  outp.write(csv_string)

In [0]:
df = pd.read_csv("imdb_master_clean.csv", index_col='Unnamed: 0')

In [0]:
unsupervised_data = df.loc[df['label'] == 'unsup']

In [0]:
unsupervised_data

,type,review,label,file
50000,train,"I admit, the great majority of films released ...",unsup,0_0.txt
50001,train,"Take a low budget, inexperienced actors doubli...",unsup,10000_0.txt
50002,train,"Everybody has seen 'Back To The Future,' right...",unsup,10001_0.txt
50003,train,Doris Day was an icon of beauty in singing and...,unsup,10002_0.txt
50004,train,"After a series of silly, fun-loving movies, 19...",unsup,10003_0.txt
...,...,...,...,...
99995,train,"Delightfully awful! Made by David Giancola, a ...",unsup,9998_0.txt
99996,train,"Watching Time Chasers, it obvious that it was ...",unsup,9999_0.txt
99997,train,At the beginning we can see members of Troma t...,unsup,999_0.txt
99998,train,"The movie was incredible, ever since I saw it ...",unsup,99_0.txt


In [0]:
%ls

assignment_5.ipynb         download.1             imdb_master.csv
cnn.ipynb                  download.2             imdb-review-dataset.zip
dist_representation.ipynb  images/                Tweets.csv
download                   imdb_master_clean.csv  Tweets.zip


In [0]:
import spacy


spacy_en = spacy.load('en')

def tokenizer(text): # create a tokenizer function
    return [token.lemma_ for token in spacy_en(text) if token.text.isalpha()]

In [0]:
%time
a = tokenizer('colourless green ideas sleep furiously')

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.77 µs


In [0]:
lemmatized_texts = [tokenizer(string) for string in unsupervised_data['review']]

In [0]:
lemmatized_texts[:10]

[['-PRON-',
  'admit',
  'the',
  'great',
  'majority',
  'of',
  'film',
  'release',
  'before',
  'say',
  'be',
  'just',
  'not',
  'for',
  '-PRON-',
  'of',
  'the',
  'dozen',
  'or',
  'so',
  'major',
  'silent',
  '-PRON-',
  'have',
  'view',
  'one',
  '-PRON-',
  'love',
  'the',
  'Crowd',
  'and',
  'two',
  'be',
  'very',
  'good',
  'the',
  'last',
  'Command',
  'and',
  'City',
  'Lights',
  'that',
  'latter',
  'Chaplin',
  'circa',
  '-PRON-',
  'be',
  'apprehensive',
  'about',
  'this',
  'one',
  'and',
  'humor',
  'be',
  'often',
  'difficult',
  'to',
  'appreciate',
  'uh',
  'enjoy',
  'decade',
  'later',
  '-PRON-',
  'do',
  'like',
  'the',
  'lead',
  'actor',
  'but',
  'think',
  'little',
  'of',
  'the',
  'intriguing',
  'sequence',
  'early',
  'on',
  'the',
  'guy',
  'be',
  'suppose',
  'to',
  'get',
  'de',
  'loused',
  'and',
  'for',
  'about',
  'three',
  'minute',
  'fully',
  'dress',
  'do',
  'some',
  'schtick',
  'in',
  '

In [0]:
import json

with open('lemmatized_texts.json', 'w', encoding='utf-8') as outp:
  json.dump(lemmatized_texts, outp, ensure_ascii=False, indent=4)

In [0]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

In [0]:
w2v_model = Word2Vec(lemmatized_texts, size=300, iter=10)

In [0]:
w2v_model.most_similar(['dog'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('cat', 0.656119704246521),
 ('puppy', 0.6271471977233887),
 ('pet', 0.5894310474395752),
 ('rat', 0.5377436280250549),
 ('baby', 0.5357066988945007),
 ('monkey', 0.530704140663147),
 ('bird', 0.5196627974510193),
 ('chicken', 0.5048027634620667),
 ('sheep', 0.4996911287307739),
 ('pig', 0.49779921770095825)]

In [0]:
w2v_model.save('w2v_unsupervised_embedding_lemmatized_space_300.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset, Iterator

In [0]:
LabelField.dtypes

{torch.float16: float,
 torch.float32: float,
 torch.float64: float,
 torch.int16: int,
 torch.int32: int,
 torch.int64: int,
 torch.int8: int,
 torch.uint8: int}

In [0]:
import torch as tt

In [0]:
from torchtext import Vocab

ImportError: ignored

## Training model

### Preparing dataset

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("imdb_master_clean.csv", index_col='Unnamed: 0')

In [0]:
df_test = df[(df['type']=='test')&(df['label']!='unsup')]
df_test.to_csv('imdb_test.csv')

In [0]:
df_train = df[(df['type']=='train')&(df['label']!='unsup')]
df_train.to_csv('imdb_train.csv')

In [0]:
df_test.head()

,type,review,label,file
0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt


### Loading dataset

In [0]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import nltk
import gensim
import spacy

from sklearn import metrics

import torch as tt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchtext.data import Field, LabelField, BucketIterator, TabularDataset, Iterator

classes = {
    'neg': 0,
    'pos': 1
}

spacy_en = spacy.load('en')

def tokenizer(text): # create a tokenizer function
    return [token.lemma_ for token in spacy_en(text) if token.text.isalpha()]

SEED = 42
np.random.seed(SEED)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
TEXT = Field(include_lengths=True, batch_first=True, 
             tokenize=tokenizer,
             eos_token='<eos>',
             lower=True,
             stop_words=nltk.corpus.stopwords.words('english'))
LABEL = LabelField(dtype=tt.int64, use_vocab=True, preprocessing=lambda x: classes[x])

test_dataset = TabularDataset('imdb_test.csv', format='csv',
                              fields = [(None,None),(None,None),('text', TEXT),('label', LABEL),(None,None)],
                              skip_header=True)

train_dataset = TabularDataset('imdb_train.csv', format='csv',
                              fields = [(None,None),(None,None),('text', TEXT),('label', LABEL),(None,None)],
                              skip_header=True)

In [0]:
train_dataset

In [0]:
model = gensim.models.Word2Vec.load('w2v_unsupervised_embedding_lemmatized_space_300.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
model.wv.save_word2vec_format('w2v_lemma_embeddings.txt')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
from torchtext.vocab import Vectors

In [0]:
vectors = Vectors(name='w2v_lemma_embeddings.txt', cache='/')

In [0]:
TEXT.build_vocab(train_dataset, min_freq=5, vectors=vectors)

In [0]:
LABEL.build_vocab(train_dataset)

### Defining the model:

In [0]:
class MyModel(nn.Module):
    
    def __init__(self, vocab_size, embed_size, hidden_size1=128, hidden_size2=16, kernels=[2,3,4,5,6], dropout_rate1=0.2, dropout_rate2=0.2, activation=nn.ReLU()):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size).from_pretrained(TEXT.vocab.vectors)
        
        self.convs = nn.ModuleList([nn.Conv1d(embed_size, hidden_size1, k, padding=5) for k in kernels])
        
        self.dropout1 = nn.Dropout(p=dropout_rate1)
        
        self.fc1 = nn.Linear(hidden_size1 * len(kernels), hidden_size2)

        self.activation = activation

        self.dropout2 = nn.Dropout(p=dropout_rate2)

        self.fc2 = nn.Linear(hidden_size2, 2)
        
    def forward(self, x):
        
        x = self.embedding(x)
        x = x.transpose(1,2)
        
        concatenated = []
        for conv in self.convs:
            z = conv(x)
            z = F.avg_pool1d(z, kernel_size=z.size(2))
            z = z.squeeze(2)
            concatenated.append(z)
            
        x = tt.cat(concatenated, 1)
        x = self.dropout1(x)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [0]:
from tqdm import tqdm_notebook

In [0]:
def train(model, train_loader, val_loader, n_batches, optimizer, criterion, n_epochs=20, 
             device=tt.device('cpu'),
             mu=0.9, 
             logdir=None,
             checkdir=None,
             reduce_lr_patience=2,
             verbose=True
            ):
    if logdir:
        sw = SummaryWriter(logdir)
    else:
        sw = None
        
    history = []
    
    if reduce_lr_patience > 0:
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=reduce_lr_patience, verbose=verbose)
    else:
        scheduler = None
    
    for epoch in range(n_epochs):
        
        # training mode
        model.train()
        running_loss = 0
        
        if verbose:
            batch_iter = tqdm_notebook(enumerate(train_loader), total=n_batches, desc='epoch %d' % (epoch + 1), leave=True)
        else:
            batch_iter = enumerate(train_loader)
            
        for i, batch in batch_iter:
            X = batch.text[0].to(device)
            y = batch.label.to(device)
            
            optimizer.zero_grad()
            
            prediction = model(X)
            
            loss = criterion(prediction, y)
            
            loss.backward()
            optimizer.step()
            
            current_loss = loss.data.detach().item()
            # smooth loss
            running_loss = running_loss * mu + current_loss * (1-mu)
            
            if verbose:
                batch_iter.set_postfix(loss='%.4f' % running_loss)
                
            niter = epoch * n_batches + i
            
            if sw:
                sw.add_scalar('Train/Loss', current_loss, niter)
                
                
        # validation mode 
        model.eval()
        val_loss = []

        with tt.no_grad():
            for batch in val_loader:
                X = batch.text[0].to(device)
                y = batch.label.to(device)

                prediction = model(X)
                loss = criterion(prediction, y)
                loss = loss.data.detach().item()
                val_loss.append( loss )

        val_loss = np.mean(val_loss)

        if verbose:
            print('validation loss=%.4f' % val_loss)

        if sw:
            sw.add_scalar('Validation/Loss', val_loss, epoch)
        
        if scheduler:
          if reduce_lr_patience > 0:
              scheduler.step(val_loss)

        if checkdir:
            torch.save(model.state_dict(), checkdir + '/epoch_%d_val_loss_%f' % (epoch, val_loss))


        history.append({
            'epoch': epoch,
            'train_loss': running_loss,
            'val_loss': val_loss,
        })

    return model, pd.DataFrame(history)

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
def eval(model, test_iterator, device, metric=accuracy_score, verbose=True):
  if verbose:
    test_iterator = tqdm_notebook(test_iterator)

  model.eval()

  scores = []

  with tt.no_grad():
    for batch in test_iterator:
      X = batch.text[0].to(device)
      y = batch.label
      y_pred = tt.argmax(model(X), dim=1)
      scores.append(accuracy_score(y, y_pred.cpu()))
  
  return np.mean(scores)

## Training the model:

In [0]:
#train_dataset, val_dataset = train_dataset.split(0.8, stratified=True)

In [0]:
# test on cpu if no errors turn on CUDA:
device = tt.device('cuda')

tt.cuda.empty_cache()

##device.empty_cache()

batch_size = 128

model = MyModel(len(TEXT.vocab.itos),
                embed_size=300,
                dropout_rate1 = 0.1,
                dropout_rate2 = 0.1).to(device)

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, val_dataset, test_dataset),
    batch_sizes=(batch_size, batch_size, batch_size),
    shuffle=True, sort=False,
#     sort_within_batch=True,
)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss().to(device)

In [0]:
model, history = train(model, train_iterator, valid_iterator, len(train_iterator), optimizer, criterion, 
                       n_epochs=11,
                       reduce_lr_patience=5,
                       device=device)

validation loss=0.5121


validation loss=0.4009


validation loss=0.3805


validation loss=0.4086


validation loss=0.3686


...

validation loss=0.3591


validation loss=0.3648


validation loss=0.3650


validation loss=0.3521


validation loss=0.3548


In [0]:
 eval(model, test_iterator, device)

0.8612244897959185